In [1]:

z

Saving Windows_log_labeled.csv to Windows_log_labeled.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/gdrive/MyDrive/W266_Final_Project/Tbird parsed/')
df=pd.read_csv('clean_log.csv')

In [2]:
!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.8.2 --quiet
!pip install transformers --quiet

     |████████████████████████████████| 24.2 MB 1.5 MB/s 
     |████████████████████████████████| 4.9 MB 28.0 MB/s 
     |████████████████████████████████| 497.9 MB 35 kB/s 
     |████████████████████████████████| 462 kB 55.9 MB/s 
     |████████████████████████████████| 5.8 MB 44.7 MB/s 
     |████████████████████████████████| 1.4 MB 58.2 MB/s 
     |████████████████████████████████| 5.5 MB 30.9 MB/s 
     |████████████████████████████████| 163 kB 61.8 MB/s 
     |████████████████████████████████| 7.6 MB 29.6 MB/s 


In [3]:
import pandas as pd
import numpy as np
import sys
import os
import re

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from transformers import BertTokenizer, TFBertModel


import sklearn as sk

import nltk
from nltk.data import find

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


#This continues to work with gensim 3.8.3.  It doesn't yet work with 4.x.  
#Make sure your pip install command specifies gensim==3.8.3
import gensim

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov  9 03:37:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
df=pd.read_csv('Windows_log_labeled.csv')

In [6]:
df.head()

,Unnamed: 0,LineId,Date,Time,Type,Admin,Content,EventId,EventTemplate,Label
0,0,1,9/27/2016,4:30:30,Info,CBS,Starting TrustedInstaller initialization.,4351fc0e,<*> TrustedInstaller <*>,0
1,1,2,9/28/2016,4:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,b642288b,Loaded Servicing Stack v<*> with Core: C:\Wind...,0
2,2,3,9/28/2016,4:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0
3,3,4,9/28/2016,4:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0
4,4,5,9/28/2016,4:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0


In [7]:
nltk.download('word2vec_sample')

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))

model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [9]:
whole_data_emb = df["Content"].values.tolist()
whole_data_label=df["Label"].values.tolist()

In [10]:
len(whole_data_emb)

79966

In [11]:
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(whole_data_emb, whole_data_label, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [12]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10



X_train, X_test, y_train, y_test = train_test_split(whole_data_emb, whole_data_label, test_size=test_ratio, random_state=1)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio))

print("Train Size:",len(X_train))
print("Test Size:",len(X_test))
print("Valid Size:",len(X_valid))

Train Size: 63972
Test Size: 7997
Valid Size: 7997


In [13]:
#@title Embedding Matrix Creation

# EMBEDDING_DIM = len(model['university'])      # we know... it's 300

# # initialize embedding matrix and word-to-id map:
# embedding_matrix = np.zeros((len(model.vocab.keys()) + 1, EMBEDDING_DIM))       
# vocab_dict = {}

# # build the embedding matrix and the word-to-id map:
# for i, word in enumerate(model.vocab.keys()):
#     embedding_vector = model[word]
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
#         vocab_dict[word] = i

# # we can use the last index at the end of the vocab for unknown tokens
# vocab_dict['[UNK]'] = len(vocab_dict)

In [14]:
SEQUENCE_LENGTH = 100

In [15]:
# def docs_to_vocab_ids(tokenized_texts_list):
#     """
#     converting a list of strings to a list of lists of word ids
#     """
#     texts_vocab_ids = []
#     text_labels = []
#     valid_example_list = []
#     for i, token_list in enumerate(tokenized_texts_list):

#         # Get the vocab id for each token in this doc ([UNK] if not in vocab)
#         vocab_ids = []
#         for token in list(token_list.numpy()):
#             decoded = token.decode('utf-8', errors='ignore')
#             if decoded in vocab_dict:
#                 vocab_ids.append(vocab_dict[decoded])
#             else:
#                 vocab_ids.append(vocab_dict['[UNK]'])
            
#         # Truncate text to max length, add padding up to max length
#         vocab_ids = vocab_ids[:SEQUENCE_LENGTH]
#         n_padding = (SEQUENCE_LENGTH - len(vocab_ids))
#         # For simplicity in this model, we'll just pad with unknown tokens
#         vocab_ids += [vocab_dict['[UNK]']] * n_padding
#         valid_example_list.append(i)
#         # Add this example to the list of converted docs
#         texts_vocab_ids.append(vocab_ids)
            
#         if i % 5000 == 0:
#             print('Examples processed: ', i)

#     print('Total examples: ', i)
#     return (np.array(texts_vocab_ids), valid_example_list)

In [16]:
# tokenizer = tf_text.WhitespaceTokenizer()
# train_tokens = tokenizer.tokenize(X_train)
                            

In [17]:
# train_valid_example_list = docs_to_vocab_ids(train_tokens)

In [18]:
# X_train[0:3]

In [19]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [21]:
num_train_examples = len(X_train)
num_test_examples = len(X_test)
num_val_examples=len(X_valid)
max_length = 25

In [22]:
#num_train_examples = 250000
num_train_examples = len(X_train)
num_test_examples = len(X_test)
num_val_examples=len(X_valid)
max_length = 25


x_train = bert_tokenizer(X_train[:num_train_examples], 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

x_test = bert_tokenizer(X_test[:num_test_examples], 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

x_val= bert_tokenizer(X_valid[:num_val_examples], 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')





def select_min_length_examples(x_data, y_data):

  x_input_ids = []
  y_labels = []

  for ((input_ids, masks), label) in zip(zip(x_data['input_ids'], x_data['attention_mask']), y_data):
    if masks[-1] == 1:
      x_input_ids.append(input_ids)
      y_labels.append(label)

  return np.array(x_input_ids), np.array(y_labels) 


In [23]:
y_train = y_train[:num_train_examples]
y_test = y_train[:num_train_examples]
y_val = y_train[:num_val_examples]

In [24]:
x_train

{'input_ids': <tf.Tensor: shape=(63972, 25), dtype=int32, numpy=
array([[  101,   143, 17239, ...,  5957,   168,   102],
       [  101,   138,  8661, ...,   118,   172,   102],
       [  101, 18012,   131, ..., 11549,   119,   102],
       ...,
       [  101,   138,  8661, ...,   118,   171,   102],
       [  101,   138,  8661, ...,   118,   188,   102],
       [  101,   138,  8661, ...,   182,   119,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(63972, 25), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(63972, 25), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=in

In [25]:
x_val

{'input_ids': <tf.Tensor: shape=(7997, 25), dtype=int32, numpy=
array([[  101,   138,  8661, ...,   118,   171,   102],
       [  101,   138,  8661, ..., 16480,  1559,   102],
       [  101,   138,  8661, ...,   118,   171,   102],
       ...,
       [  101,   138,  8661, ...,   118,   181,   102],
       [  101,   138,  8661, ..., 16480,  1559,   102],
       [  101,   138,  8661, ...,   172,   119,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(7997, 25), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(7997, 25), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32

In [26]:
bert_train_input_ids, bert_train_labels = select_min_length_examples(x_train, y_train)
bert_test_input_ids, bert_test_labels = select_min_length_examples(x_test, y_test)

In [27]:
bert_val_input_ids, bert_val_labels = select_min_length_examples(x_val, y_val)

In [28]:
SEQUENCE_LENGTH = 25

In [29]:
def create_bert_cls_model(hidden_size = 100, 
                          dropout=0.3,
                          learning_rate=0.005):
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """


    

    ### YOUR CODE HERE
    input_ids = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    #token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    #attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    
    bert_inputs = {'input_ids': input_ids}  

 

    bert_out = bert_model(bert_inputs)

    pooled_token = bert_out[1]
    #cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooled_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')


    ### END YOUR CODE
    print(classification_model.summary())
    return classification_model

In [30]:
bert_classification_model = create_bert_cls_model()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids_layer (InputLayer  [(None, 25)]             0         
 )                                                               
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             25, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                      

In [31]:
b_model = create_bert_cls_model()
history_wan = b_model.fit(bert_train_input_ids,
                            bert_train_labels,
                            validation_data=(bert_val_input_ids,bert_val_labels),
                            batch_size=8,
                            epochs=2
                            )

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids_layer (InputLayer  [(None, 25)]             0         
 )                                                               
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             25, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                    